In [1]:
import torch
import os
import sys
import random

# custom
from util import *
from AAC_Prefix.AAC_Prefix import * # network
from Train import *
    
TEST_BATCH_SIZE = 5

USE_CUDA = torch.cuda.is_available() 
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

In [11]:
# table_num = 1 : Evaluation on Clotho
# table_num = 2 : Evaluation on AudioCaps

# setting_num = 1 : train dataset == test dataset
# setting_num = 2 : train dataset != test dataset
# setting_num = 3 : overall datasets(Clotho & AudioCaps) <- need to test by using compressed audio

table_num = 1
setting_num = 1

if setting_num == 3 :
    is_settingnum_3 = True
else : 
    is_settingnum_3 = False

model = get_model_in_table(table_num, setting_num, device)

/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=512 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


use Custom Tokenizer
temporal feature's mapping network : num_head = 8 num_layers = 4
global feature ver's mapping network : num_head = 8 num_layers = 4
Encoder freezing
GPT2 freezing


In [4]:
test_dataloader_audiocaps  = CreateDataloader(None, './AudioCaps', TEST_BATCH_SIZE, 'test', None, is_TrainDataset = False, tokenizer_type = None, is_settingnum_3 = is_settingnum_3)
test_dataloader_clotho = CreateDataloader(None, './Clotho', TEST_BATCH_SIZE, 'evaluation', None, is_TrainDataset = False, tokenizer_type = None, is_settingnum_3 = is_settingnum_3)

get dataset...: 100%|██████████████████████| 1045/1045 [00:06<00:00, 166.82it/s]


In [12]:
if table_num == 1 or setting_num == 3 :
    metrics_clotho, captions_pred_clotho, captions_gt_clotho = eval_model(model, test_dataloader_clotho, 31, 'test', True, device, 'Clotho')

Eval using dataset...: 100%|████████████████| 1045/1045 [03:32<00:00,  4.92it/s]


loading annotations into memory...
0:00:00.006926
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 70696 tokens at 477904.72 tokens per second.
PTBTokenizer tokenized 11930 tokens at 143140.25 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 9840, 'reflen': 10288, 'guess': [9840, 8795, 7750, 6705], 'correct': [5772, 2326, 925, 286]}
ratio: 0.9564541213062834
Bleu_1: 0.560
Bleu_2: 0.376
Bleu_3: 0.253
Bleu_4: 0.160
computing METEOR score...
METEOR: 0.170
computing Rouge score...
ROUGE_L: 0.378
computing CIDEr score...
CIDEr: 0.392
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.4 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.3 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.6

SPICE evaluation took: 10.78 s
SPICE: 0.118
computing SPIDEr score...
SPIDEr: 0.255


In [13]:
if table_num == 2 or setting_num == 3 :
    metrics_audiocaps, captions_pred_audiocaps, captions_gt_audiocaps = eval_model(model, test_dataloader_audiocaps, 31, 'test', True, device, 'AudioCaps')